# 20241003

0. Oncell data이기때문에,  X_reg값을 이용하면 됨. (ADI처럼 KMRC Decorrect하고, MRC_X를 더하는 처리를 할 필요가 없다.)

1. K MRC Decorrect 과정은 skip합니다.  (oncell이라서 KMRC Input값 없음)

2. -PSM + PointMRC 처리는 skip합니다. (oncell이라서 pointmrc는 값이 없을것이고, psm만 빼주려면 )

3. OSR 처리는 합니다. -> oncell residual 확보. 

4. residual 에서 PSM Input을 decorrect 해야함. 
★ PSM Input을 decorrect 처리방식을 결정해야함 
방법1. PSM RK값을 Fitting해서 구한다음에 residual에서 빼는 방법. 
방법2. MRC_X를 이용하는 방법.  Oncell data에는 Point MRC가 없을거라서, MRC_X가 곧 PSM Input(부호는 반대)을 의미함 
       -> residual에서 MRC_X만큼을 더해주면 됌. 


우선 방법1로 해봅니다.
1. PSM시트에서 RK값 끌어와서 Fitting. 
2. De-PSM.  ( Oncell Residual에서 PSM Fitting 빼주기 )   residual_x - psm_pred_x = residual_x_depsm 
3. residual_x_depsm 을 cpe 38para로 모델링. -> ideal_psm 
4. ideal_psm의 변동을 확인. -> wafer map & m3s값 trend 



# 20241003(2)

psm input이 없는 shot에서는 해당샷을 skip처리해버리고 있음.  ->  no psm shot에 대해서는 0으로 처리해서 계산에 포함시키게 변경

# 20241003(3)
residual_x_depsm 을 cpe 38para로 모델링. -> ideal_psm



In [1]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
import openpyxl
import time
from datetime import datetime


# nau 파일이 있는 폴더 경로
folder_path = 'C:/py_data/nau'


################################################ NAU파일제목에서 M_STEP정보 끌어와서 시트에 저장  ########################################

# 폴더 내의 모든 파일에 대해 실행
for file_name in os.listdir(folder_path):
    # 파일 경로 설정
    file_path = os.path.join(folder_path, file_name)
    
    # 파일인지 확인 (폴더는 제외)
    if os.path.isfile(file_path) and file_name.endswith('.nau'):
        try:
            # 파일 확장자를 임시로 .xlsx로 변경
            temp_file_path = file_path.replace('.nau', '.xlsx')
            os.rename(file_path, temp_file_path)

            # pandas를 이용해 엑셀 파일(유사)을 읽기
            excel_data = pd.read_excel(temp_file_path, engine='openpyxl')
            print(f"{file_name} 파일 읽기 성공")

            # 파일명을 "_" 기준으로 분할
            file_name_without_extension = os.path.splitext(file_name)[0]
            split_file_name = file_name_without_extension.split("_")

            # 기존 엑셀 파일 열기
            workbook = load_workbook(temp_file_path)

            # 파일명 시트를 첫 번째로 추가
            new_sheet_name = "FileName"
            if new_sheet_name not in workbook.sheetnames:
                new_sheet = workbook.create_sheet(title=new_sheet_name, index=0)  # 첫 번째 시트로 추가
            else:
                new_sheet = workbook[new_sheet_name]

            # 분할된 파일명을 첫 번째 시트에 기록
            for col, value in enumerate(split_file_name, start=1):
                new_sheet.cell(row=1, column=col, value=value)

            # 변경된 파일 저장
            workbook.save(temp_file_path)
            print(f"{file_name} 파일에 새로운 시트 추가 완료")

            # 확장자를 다시 .nau로 복원
            os.rename(temp_file_path, file_path)

        except Exception as e:
            print(f"{file_name} 파일을 읽는 중 에러 발생: {e}")




################################################ NAU로부터 필요데이터 추출 (전처리) ########################################




# 추출할 컬럼 위치 설정 (예: 첫 번째 열은 0, 두 번째 열은 1로 인덱스 시작)
columns_to_extract = [0, 1, 2, 3, 4, 5, 6, 7]  # Wafer, TEST, DieX, DieY, X_reg, Y_reg, MRC_X, MRC_Y의 열 위치

def process_nau_files(folder_path, columns_to_extract):
    # 결과를 담을 리스트 생성
    combined_rawdata_list = []
    combined_trocs_input_list = []
    mrc_data_list = []
    combined_psm_input_list = []

    # 폴더 내 모든 nau 파일에 대해 반복
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.nau'):
            file_path = os.path.join(folder_path, file_name)
            
            # 필요한 시트만 읽기
            rawdata_file = pd.read_excel(file_path, sheet_name='RawData-1')
            trocs_input_file = pd.read_excel(file_path, sheet_name='Trocs Input')
            psm_input_file = pd.read_excel(file_path, sheet_name='PerShotMRC')

            ########### m_step 시트 불러오기 #########################
            m_step_file = pd.read_excel(file_path, sheet_name='FileName')

            ######################################## RawData-1 전처리 ##################################################
            # 지정된 열 추출
            extracted_data_raw = rawdata_file.iloc[:, columns_to_extract].copy()

            # 'LOT_ID' 값 및 'STEPSEQ' 추출
            lot_id_value_raw = rawdata_file.columns[13]
            stepseq_value_raw = rawdata_file.iloc[0, 13]
            wafer_value_raw = rawdata_file.iloc[0, 0]
            p_eqpid__value_raw = rawdata_file.iloc[1, 13]
            photo_ppid_value_raw = rawdata_file.iloc[11, 13]
            p_time_value_raw = rawdata_file.iloc[2, 13]
            m_time_value_raw = rawdata_file.iloc[4, 13]
            chuckid_value_raw = rawdata_file.iloc[15, 13]

            # 'm_step' 값 추출 및 신규컬럼 추가
            m_step_value_raw = m_step_file.columns[1]
            extracted_data_raw['M_STEP'] = m_step_value_raw

                
            # 새로운 컬럼 추가
            extracted_data_raw['STEPSEQ'] = stepseq_value_raw
            extracted_data_raw['LOT_ID'] = lot_id_value_raw

            # 추가 정보 추출 및 컬럼 추가
            extracted_data_raw['STEP_PITCH_X'] = rawdata_file.iloc[6, 13]
            extracted_data_raw['STEP_PITCH_Y'] = rawdata_file.iloc[7, 13]
            extracted_data_raw['MAP_SHIFT_X'] = rawdata_file.iloc[8, 13]
            extracted_data_raw['MAP_SHIFT_Y'] = rawdata_file.iloc[9, 13]

            # 'coordinate_X', 'coordinate_Y' 매핑
            coord_map = rawdata_file[['Test No', 'coordinate_X', 'coordinate_Y']].drop_duplicates(subset='Test No').set_index('Test No')
            extracted_data_raw['coordinate_X'] = extracted_data_raw['TEST'].map(coord_map['coordinate_X'])
            extracted_data_raw['coordinate_Y'] = extracted_data_raw['TEST'].map(coord_map['coordinate_Y'])
                  
            # 'wf_x' 및 'wf_y' 계산
            extracted_data_raw['wf_x'] = (
                extracted_data_raw['DieX'] * extracted_data_raw['STEP_PITCH_X'] +
                extracted_data_raw['MAP_SHIFT_X'] + extracted_data_raw['coordinate_X']
            )
            extracted_data_raw['wf_y'] = (
                extracted_data_raw['DieY'] * extracted_data_raw['STEP_PITCH_Y'] +
                extracted_data_raw['MAP_SHIFT_Y'] + extracted_data_raw['coordinate_Y']
            )

            ##### context 정보추가 #####
            extracted_data_raw['P_EQPID'] = rawdata_file.iloc[1, 13]
            extracted_data_raw['P_TIME'] = rawdata_file.iloc[2, 13]
            extracted_data_raw['M_TIME'] = rawdata_file.iloc[4, 13]
            extracted_data_raw['Photo_PPID'] = rawdata_file.iloc[11, 13]
            extracted_data_raw['Base_EQP1'] = rawdata_file.iloc[12, 13]
            extracted_data_raw['ChuckID'] = rawdata_file.iloc[15, 13]
            extracted_data_raw['ReticleID'] = rawdata_file.iloc[16, 13]
            
            

            ######## 'Unique_ID'라는 새로운 컬럼 추가   ########  
            ## M_TIME 추가(240922)

            extracted_data_raw['UNIQUE_ID'] = extracted_data_raw.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)


            # 'Unique_ID2 추가 (TEST, DieX, DieY 추가. 정렬용)
            extracted_data_raw['UNIQUE_ID2'] = extracted_data_raw.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['TEST']}_{row['DieX']}_{row['DieY']} ", axis=1)
                      
            # 컬럼 순서 재조정
            cols_order = [
                'UNIQUE_ID', 'UNIQUE_ID2',
                'STEPSEQ', 'LOT_ID', 'Wafer', 
                'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_STEP', 'M_TIME', 'ChuckID', 'ReticleID', 'Base_EQP1', 
                'TEST', 'DieX', 'DieY',
                'X_reg', 'Y_reg', 'MRC_X', 'MRC_Y', 
                'STEP_PITCH_X', 'STEP_PITCH_Y', 'MAP_SHIFT_X', 'MAP_SHIFT_Y', 'coordinate_X', 'coordinate_Y', 'wf_x', 'wf_y'
            ]
            extracted_data_raw = extracted_data_raw[cols_order]

            # 리스트에 추가
            combined_rawdata_list.append(extracted_data_raw)

            ######################################## Trocs Input 전처리 ##################################################

            trocs_input_file['STEPSEQ'] = stepseq_value_raw
            trocs_input_file['LOT_ID'] = lot_id_value_raw
            trocs_input_file['Wafer'] = wafer_value_raw
            trocs_input_file['P_EQPID'] = p_eqpid__value_raw
            trocs_input_file['Photo_PPID'] = photo_ppid_value_raw
            trocs_input_file['P_TIME'] = p_time_value_raw 
            trocs_input_file['M_TIME'] = m_time_value_raw 
            trocs_input_file['ChuckID'] = chuckid_value_raw
            
            # 'm_step' 신규컬럼 추가
            trocs_input_file['M_STEP'] = m_step_value_raw 

         
            # Trocs Input 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            trocs_input_file['UNIQUE_ID'] = trocs_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)

            # 'Unique_ID2 추가 (DieX(=dCol), DieY(=dRow) 추가. 정렬용)
            trocs_input_file['UNIQUE_ID2'] = trocs_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['dCol']}_{row['dRow']}", axis=1)
            
            # 순서 재조정
            # 기존 컬럼 뒤로 밀기 - 순서를 맨 앞으로 추가하는 순서로 재배열
            cols_to_insert = ['UNIQUE_ID', 'UNIQUE_ID2', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_STEP','M_TIME', 'ChuckID']

            # 각 컬럼을 지정된 위치에 삽입 (맨 앞에)
            for i, col in enumerate(cols_to_insert):
                trocs_input_file.insert(i, col, trocs_input_file.pop(col))

            # 리스트에 추가
            combined_trocs_input_list.append(trocs_input_file)

            ######################################## MRC 전처리 ##################################################
            
            # MRC 데이터 추출을 위해 'RawData-1' 시트를 header=None으로 다시 읽음
            rawdata_file_no_header = pd.read_excel(file_path, sheet_name='RawData-1', header=None)

            # MRC 데이터 추출
            mrc_part1 = rawdata_file_no_header.iloc[0:20, 15:17]
            mrc_part2 = rawdata_file_no_header.iloc[22:40, 15:17]
            mrc_part = pd.concat([mrc_part1, mrc_part2], ignore_index=True)

            # 컬럼 이름 설정
            mrc_part.columns = ['K PARA', 'GPM']

            # INDEX 컬럼 추가 (1부터 시작하는 순차적 번호)
            mrc_part['INDEX'] = range(1, len(mrc_part) + 1)


            # 컬럼 추가             
            mrc_part['STEPSEQ'] = stepseq_value_raw
            mrc_part['LOT_ID'] = lot_id_value_raw
            mrc_part['Wafer'] = wafer_value_raw
            mrc_part['P_EQPID'] = p_eqpid__value_raw
            mrc_part['Photo_PPID'] = photo_ppid_value_raw
            mrc_part['P_TIME'] = p_time_value_raw 
            mrc_part['M_TIME'] = m_time_value_raw 
            mrc_part['ChuckID'] = chuckid_value_raw

            # 'm_step' 신규컬럼 추가
            mrc_part['M_STEP'] = m_step_value_raw 

            # mrc_part 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            mrc_part['UNIQUE_ID'] = mrc_part.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)

            # 컬럼 순서 재조정
            mrc_cols_order = [
                'UNIQUE_ID',
                'STEPSEQ', 'LOT_ID', 'Wafer', 
                'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_STEP','M_TIME', 'ChuckID',  
                'K PARA', 'GPM', 'INDEX'
            ]
            mrc_part = mrc_part[mrc_cols_order]

            
            # 리스트에 추가
            mrc_data_list.append(mrc_part)

            
            ######################################## PSM 전처리 ##################################################

            psm_input_file['STEPSEQ'] = stepseq_value_raw
            psm_input_file['LOT_ID'] = lot_id_value_raw
            psm_input_file['Wafer'] = wafer_value_raw
            psm_input_file['P_EQPID'] = p_eqpid__value_raw
            psm_input_file['Photo_PPID'] = photo_ppid_value_raw
            psm_input_file['P_TIME'] = p_time_value_raw 
            psm_input_file['M_TIME'] = m_time_value_raw 
            psm_input_file['ChuckID'] = chuckid_value_raw

            # 'm_step' 신규컬럼 추가
            psm_input_file['M_STEP'] = m_step_value_raw 

            # PSM Input 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            psm_input_file['UNIQUE_ID'] = psm_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)

            # 'Unique_ID2 추가 (DieX(=dCol), DieY(=dRow) 추가. 정렬용)
            psm_input_file['UNIQUE_ID2'] = psm_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['dCol']}_{row['dRow']}", axis=1)
            
            
            # 순서 재조정
            # 기존 컬럼 뒤로 밀기 - 순서를 맨 앞으로 추가하는 순서로 재배열
            cols_to_insert_psm = ['UNIQUE_ID', 'UNIQUE_ID2', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_STEP','M_TIME', 'ChuckID']

            # 각 컬럼을 지정된 위치에 삽입 (맨 앞에)
            for i, col in enumerate(cols_to_insert_psm):
                psm_input_file.insert(i, col, psm_input_file.pop(col))

            # 리스트에 추가
            combined_psm_input_list.append(psm_input_file)



    # 리스트를 데이터프레임으로 병합
    combined_rawdata = pd.concat(combined_rawdata_list, ignore_index=True)
    combined_trocs_input = pd.concat(combined_trocs_input_list, ignore_index=True)
    mrc_data = pd.concat(mrc_data_list, ignore_index=True)
    combined_psm_input = pd.concat(combined_psm_input_list, ignore_index=True)

    # 병합 후, UNIQUE_ID기준으로 오름차순 정렬 (내림차순하고 싶으면 ascending=False 옵션 추가하면 됨.) 
    combined_rawdata = combined_rawdata.sort_values(by=['UNIQUE_ID', 'TEST', 'DieX', 'DieY'])    
    combined_trocs_input = combined_trocs_input.sort_values(by=['UNIQUE_ID', 'dCol', 'dRow'])
    mrc_data = mrc_data.sort_values(by=['UNIQUE_ID', 'INDEX'])
    combined_psm_input = combined_psm_input.sort_values(by=['UNIQUE_ID', 'dCol', 'dRow'])


    # 최종 데이터를 엑셀 파일로 저장
    with pd.ExcelWriter('output.xlsx') as writer:
        combined_rawdata.to_excel(writer, sheet_name='RawData-1', index=False)
        combined_trocs_input.to_excel(writer, sheet_name='Trocs Input', index=False)
        mrc_data.to_excel(writer, sheet_name='MRC', index=False)
        combined_psm_input.to_excel(writer, sheet_name='PerShotMRC', index=False)






###################### OSR ###########################



def multi_lot_regression(df_rawdata):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')

    # 회귀분석 결과를 저장할 리스트
    wkrk_results = []

    # 각 그룹에 대해 처리
    for unique_id, group in grouped:
        die_x = group['DieX']
        die_y = group['DieY']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        field_x = group['coordinate_X']
        field_y = group['coordinate_Y']
        
        # 좌표 계산
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = field_x
        ry = field_y

        # X_dx, X_dy 행렬 구성
        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 종속변수
        Y_dx = group['X_reg']
        Y_dy = group['Y_reg']

        # 최소자승법으로 계수 계산
        coeff_dx = np.linalg.lstsq(X_dx, Y_dx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(X_dy, Y_dy, rcond=None)[0]

        # 결과 저장
        wkrk_results.append(pd.DataFrame({
            'UNIQUE_ID': [unique_id],
            'WK1': [coeff_dx[0]],
            'WK2': [coeff_dy[0]],
            'WK3': [coeff_dx[1]],
            'WK4': [coeff_dy[1]],
            'WK5': [coeff_dx[2]],
            'WK6': [coeff_dy[2]],
            'WK7': [coeff_dx[3]],
            'WK8': [coeff_dy[3]],
            'WK9': [coeff_dx[4]],
            'WK10': [coeff_dy[4]],
            'WK11': [coeff_dx[5]],
            'WK12': [coeff_dy[5]],
            'WK13': [coeff_dx[6]],
            'WK14': [coeff_dy[6]],
            'WK15': [coeff_dx[7]],
            'WK16': [coeff_dy[7]],
            'WK17': [coeff_dx[8]],
            'WK18': [coeff_dy[8]],
            'WK19': [coeff_dx[9]],
            'WK20': [coeff_dy[9]],
            'RK1': [0],
            'RK2': [0],
            'RK3': [coeff_dx[10]],
            'RK4': [coeff_dy[10]],
            'RK5': [coeff_dx[11]],
            'RK6': [coeff_dy[11]],
            'RK7': [coeff_dx[12]],
            'RK8': [coeff_dy[12]],
            'RK9': [coeff_dx[13]],
            'RK10': [coeff_dy[13]],
            'RK11': [coeff_dx[14]],
            'RK12': [coeff_dy[14]],
            'RK13': [coeff_dx[15]],
            'RK14': [coeff_dy[15]],
            'RK15': [coeff_dx[16]],
            'RK16': [coeff_dy[16]],
            'RK17': [coeff_dx[17]],
            'RK18': [coeff_dy[17]],
            'RK19': [coeff_dx[18]],
            'RK20': [0],
        }))

    # 결과 병합
    combined_results = pd.concat(wkrk_results, ignore_index=True)
    return combined_results






def multi_lot_fitting_residual(df_rawdata, df_coeff):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')
    
    # 예측 결과를 저장할 리스트
    predictions_list = []
    
    for unique_id, group in grouped:
        stepseq = group['STEPSEQ']
        lot_id = group['LOT_ID']
        wafer = group['Wafer']
        p_eqpid = group['P_EQPID']
        photo_ppid = group['Photo_PPID']
        p_time = group['P_TIME']
        m_step = group['M_STEP']
        m_time = group['M_TIME']
        chuckid = group['ChuckID']
        reticleid = group['ReticleID']
        base_eqp1 = group['Base_EQP1']
        test = group['TEST']
        die_x = group['DieX']
        die_y = group['DieY']
        mrc_x = group['MRC_X']
        mrc_y = group['MRC_Y']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        coordinate_x = group['coordinate_X']
        coordiante_y = group['coordinate_Y']   
        wf_x = group['wf_x']
        wf_y = group['wf_y']        



        x_reg = group['X_reg']
        y_reg = group['Y_reg']

      
        
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = coordinate_x
        ry = coordiante_y

        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 해당 LOT_ID의 계수 추출
        coeffs = df_coeff[df_coeff['UNIQUE_ID'] == unique_id].iloc[0]
        coeff_dx = coeffs[['WK1','WK3','WK5','WK7','WK9','WK11','WK13','WK15','WK17','WK19','RK3','RK5','RK7','RK9','RK11','RK13','RK15','RK17','RK19']].values.astype(float)
        coeff_dy = coeffs[['WK2','WK4','WK6','WK8','WK10','WK12','WK14','WK16','WK18','WK20','RK4','RK6','RK8','RK10','RK12','RK14','RK16','RK18']].values.astype(float)


        # 예측값 계산
        pred_x = X_dx.dot(coeff_dx)
        pred_y = X_dy.dot(coeff_dy)

        # 잔차 계산
        residual_x = group['X_reg'] - pred_x
        residual_y = group['Y_reg'] - pred_y

        # 결과 저장
        predictions_list.append(pd.DataFrame({
            'UNIQUE_ID' : unique_id,
            'STEPSEQ' : stepseq, 
            'LOT_ID' : lot_id,
            'Wafer' : wafer, 
            'P_EQPID' : p_eqpid,
            'Photo_PPID' : photo_ppid,
            'P_TIME' : p_time,
            'M_STEP' : m_step,
            'M_TIME' : m_time,
            'ChuckID' : chuckid,
            'ReticleID' : reticleid,
            'Base_EQP1' : base_eqp1,
            'TEST' : test,
            'DieX' : die_x,
            'DieY' : die_y,
            'MRC_X' : mrc_x,
            'MRC_Y' : mrc_y,
            'STEP_PITCH_X' : step_pitch_x,
            'STEP_PITCH_Y' : step_pitch_y,
            'MAP_SHIFT_X' : map_shift_x,
            'MAP_SHIFT_Y' : map_shift_y,
            'coordinate_X' : coordinate_x,
            'coordinate_Y' : coordiante_y,
            'wf_x' : wf_x,
            'wf_y' : wf_y,

            'X_reg' : x_reg,
            'Y_reg' : y_reg,

            'pred_x': pred_x,
            'pred_y': pred_y,
            'residual_x': residual_x,
            'residual_y': residual_y,
        }))

    # 예측 결과 병합
    df_predictions = pd.concat(predictions_list, ignore_index=True)
    return df_predictions





###################### PSM Input Decorrect ################################ 

def psm_decorrect(df_residata, df_psm_input):
    # 'shot' 별로 데이터를 그룹화 (고유한 lot_id, die_x, die_y 조합)
    grouped = df_residata.groupby(['UNIQUE_ID', 'DieX', 'DieY'])

    # 예측 결과를 저장할 리스트
    psm_input_list = []

    # 각 그룹에 대해 연산 수행
    for (unique_id, diex, diey), group in grouped:

        stepseq = group['STEPSEQ']
        lot_id = group['LOT_ID']
        wafer = group['Wafer']
        p_eqpid = group['P_EQPID']
        photo_ppid = group['Photo_PPID']
        p_time = group['P_TIME']
        m_step = group['M_STEP']
        m_time = group['M_TIME']
        chuckid = group['ChuckID']
        reticleid = group['ReticleID']
        base_eqp1 = group['Base_EQP1']
        test = group['TEST']
        die_x = group['DieX']
        die_y = group['DieY']
        mrc_x = group['MRC_X']
        mrc_y = group['MRC_Y']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        coordinate_x = group['coordinate_X']
        coordinate_y = group['coordinate_Y']   
        wf_x = group['wf_x']
        wf_y = group['wf_y']  

        x_reg = group['X_reg']
        y_reg = group['Y_reg']
        pred_x = group['pred_x']
        pred_y = group['pred_y']
        residual_x = group['residual_x']
        residual_y = group['residual_y']

        
        # 독립변수 설정 ('coordinate_X', 'coordinate_Y'를 독립변수로 사용)
        rx = group['coordinate_X'].values
        ry = group['coordinate_Y'].values

        # 독립 변수 배열구성
        X_dx = np.vstack([
            np.ones(len(rx)),
            (rx)/1e6,     (-ry)/1e6, 
            (rx**2)/1e9,  (rx*ry)/1e9,     (ry**2)/1e9,
            (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12,    (ry**3)/1e12,
            (rx**4)/1e19, (rx**3*ry)/1e19, (rx**2*ry**2)/1e19, (rx*ry**3)/1e19,    (ry**4)/1e19,
            (rx**5)/1e23, (rx**4*ry)/1e23, (rx**3*ry**2)/1e23, (rx**2*ry**3)/1e23, (rx*ry**4)/1e23,    (ry**5)/1e23,
            (rx**6)/1e27, (rx**5*ry)/1e27, (rx**4*ry**2)/1e27, (rx**3*ry**3)/1e27, (rx**2*ry**4)/1e27, (rx*ry**5)/1e27,    (ry**6)/1e27,
            (rx**7)/1e31, (rx**6*ry)/1e31, (rx**5*ry**2)/1e31, (rx**4*ry**3)/1e31, (rx**3*ry**4)/1e31, (rx**2*ry**8)/1e31, (rx*ry**6)/1e31, (ry**7)/1e31
        ]).T

        X_dy = np.vstack([
            np.ones(len(ry)), 
            (ry)/1e6,     (rx)/1e6,
            (ry**2)/1e9,  (ry*rx)/1e9,     (rx**2)/1e9,
            (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12,    (rx**3)/1e12,
            (ry**4)/1e19, (ry**3*rx)/1e19, (ry**2*rx**2)/1e19, (ry*rx**3)/1e19,    (rx**4)/1e19,
            (ry**5)/1e23, (ry**4*rx)/1e23, (ry**3*rx**2)/1e23, (ry**2*rx**3)/1e23, (ry*rx**4)/1e23,    (rx**5)/1e23,
            (ry**6)/1e27, (ry**5*rx)/1e27, (ry**4*rx**2)/1e27, (ry**3*rx**3)/1e27, (ry**2*rx**4)/1e27, (ry*rx**5)/1e27,    (rx**6)/1e27,
            (ry**7)/1e31, (ry**6*rx)/1e31, (ry**5*rx**2)/1e31, (ry**4*rx**3)/1e31, (ry**3*rx**4)/1e31, (ry**2*rx**8)/1e31, (ry*rx**6)/1e31, (rx**7)/1e31    
            
        ]).T


        # 종속변수 설정 ( PSM INPUT의 RK값을 독립변수로 사용)
        # PerShotMRC 시트에서 해당 UNIQUE_ID에 해당하는 rk1~rk72 값을 추출
        psm_row = df_psm_input[(df_psm_input['UNIQUE_ID'] == unique_id) & (df_psm_input['dCol'] == diex) & (df_psm_input['dRow'] == diey)]

        if psm_row.empty:
            # PSM input이 없는 경우 0으로 처리
            Y_dx = np.zeros(36)  # 홀수 rk값의 수만큼 0 배열 생성
            Y_dy = np.zeros(36)  # 짝수 rk값의 수만큼 0 배열 생성
        else:
            # 홀수 rk 값 (Y_dx)과 짝수 rk 값 (Y_dy) 추출
            rk_values = psm_row.iloc[:, 15:87]  # rk1 ~ rk72 열을 선택      

            # Y_dx는 rk1부터 rk71까지 홀수 열을 추출
            Y_dx = rk_values.iloc[:, ::2].values.flatten()       

            # Y_dy는 rk2부터 rk72까지 짝수 열을 추출
            Y_dy = rk_values.iloc[:, 1::2].values.flatten()  


        # 행렬 곱을 통해 예측 값 계산
        psm_fit_x = X_dx.dot(Y_dx)
        psm_fit_y = X_dy.dot(Y_dy)       

        residual_x_depsm = group['residual_x'] - psm_fit_x
        residual_y_depsm = group['residual_y'] - psm_fit_y

        # 결과 저장
        psm_input_list.append(pd.DataFrame({
            'UNIQUE_ID': unique_id,
            'STEPSEQ' : stepseq, 
            'LOT_ID' : lot_id,
            'Wafer' : wafer, 
            'P_EQPID' : p_eqpid,
            'Photo_PPID' : photo_ppid,
            'P_TIME' : p_time,
            'M_STEP' : m_step,
            'M_TIME' : m_time,
            'ChuckID' : chuckid,
            'ReticleID' : reticleid,
            'Base_EQP1' : base_eqp1,
            'TEST' : test,
            'DieX' : die_x,
            'DieY' : die_y,
            'MRC_X' : mrc_x,
            'MRC_Y' : mrc_y,
            'STEP_PITCH_X' : step_pitch_x,
            'STEP_PITCH_Y' : step_pitch_y,
            'MAP_SHIFT_X' : map_shift_x,
            'MAP_SHIFT_Y' : map_shift_y,
            'coordinate_X' : coordinate_x,
            'coordinate_Y' : coordinate_y,
            'wf_x' : wf_x,
            'wf_y' : wf_y,
            'X_reg' : x_reg,
            'Y_reg' : y_reg,
            'pred_x': pred_x,
            'pred_y': pred_y,
            'residual_x': residual_x,
            'residual_y': residual_y,

            'psm_fit_x': psm_fit_x,
            'psm_fit_y': psm_fit_y,
            'residual_x_depsm' : residual_x_depsm,
            'residual_y_depsm' : residual_y_depsm
               
        }))


    
    # 결과 병합
    df_psm_de = pd.concat(psm_input_list, ignore_index=True)
    # 정렬 (★ 기존데이터의 정렬순서와 맞춰주기위한 작업)
    df_psm_de = df_psm_de.sort_values(by=['UNIQUE_ID', 'TEST', 'DieX', 'DieY'])
    return df_psm_de






###################### resi_depsm 을 CPE 19para 모델링 ################################ 



def resi_to_cpe(df_resi_depsm):
    # 'shot' 별로 데이터를 그룹화 (고유한 die_x, die_y 조합)
    grouped = df_resi_depsm.groupby(['UNIQUE_ID', 'DieX', 'DieY'])
    
    # 회귀분석 결과를 저장할 리스트
    shot_regression_results = []

    for (unique_id, die_x, die_y), group in grouped:
        # 독립변수와 종속변수 설정 
        # 독립변수 (shot 좌표)
        rx = group['coordinate_X']
        ry = group['coordinate_Y']  

        # 종속변수 (residual)
        Yx = group['residual_x_depsm']
        Yy = group['residual_y_depsm']



        # 독립 변수 배열구성 ( 38PARA )
        X_dx = np.vstack([
            np.ones(len(rx)),
            (rx)/1e6,     (-ry)/1e6, 
            (rx**2)/1e9,  (rx*ry)/1e9,     (ry**2)/1e9,
            (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12,    (ry**3)/1e12,

        ]).T

        X_dy = np.vstack([
            np.ones(len(ry)), 
            (ry)/1e6,     (rx)/1e6,
            (ry**2)/1e9,  (ry*rx)/1e9,     (rx**2)/1e9,
            (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12,    
            
        ]).T
        


        coeff_dx = np.linalg.lstsq(X_dx, Yx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(X_dy, Yy, rcond=None)[0]


        # 회귀분석 결과를 리스트에 저장
        shot_regression_results.append({
            'UNIQUE_ID': unique_id,
            'DieX': die_x,
            'DieY': die_y,
            'RK1': coeff_dx[0],
            'RK2': coeff_dy[0],
            'RK3': coeff_dx[1],
            'RK4': coeff_dy[1],
            'RK5': coeff_dx[2],
            'RK6': coeff_dy[2],

            'RK7': coeff_dx[3],
            'RK8': coeff_dy[3],
            'RK9': coeff_dx[4],
            'RK10': coeff_dy[4],
            'RK11': coeff_dx[5],
            'RK12': coeff_dy[5],
            'RK13': coeff_dx[6],
            'RK14': coeff_dy[6],
            'RK15': coeff_dx[7],
            'RK16': coeff_dy[7],
            'RK17': coeff_dx[8],
            'RK18': coeff_dy[8],
            'RK19': coeff_dx[9],
            'RK20': 0,            
        })
    
    # 회귀분석 결과를 새로운 DataFrame으로 변환
    df_cpe19p = pd.DataFrame(shot_regression_results)
    return df_cpe19p







###################### CPE 19para을 fitting (=Ideal PSM) ################################ 

def cpe38p_fitting(df_resi_depsm, df_cpe19p):
    # 결과를 저장할 리스트 생성
    predictions_list = []

    # 각 데이터 포인트에 대해 처리
    for idx, row in df_resi_depsm.iterrows():
        
        unique_id = row['UNIQUE_ID']
        die_x = row['DieX']
        die_y = row['DieY']
        coordinate_x = row['coordinate_X']
        coordinate_y = row['coordinate_Y']
        residual_x_depsm = row['residual_x_depsm']
        residual_y_depsm = row['residual_y_depsm']
        psm_fit_x = row['psm_fit_x']
        psm_fit_y = row['psm_fit_y']


        
        rx = coordinate_x
        ry = coordinate_y

        # 해당 LOT_ID, DieX, DieY에 해당하는 회귀계수 찾기
        coeffs = df_cpe19p[(df_cpe19p['UNIQUE_ID'] == unique_id) & (df_cpe19p['DieX'] == die_x) & (df_cpe19p['DieY'] == die_y)]
        
        if not coeffs.empty:
            coeffs = coeffs.iloc[0]
            # 회귀계수 추출
            RK1 = coeffs['RK1']
            RK2 = coeffs['RK2']
            RK3 = coeffs['RK3']
            RK4 = coeffs['RK4']
            RK5 = coeffs['RK5']
            RK6 = coeffs['RK6']
            RK7 = coeffs['RK7']
            RK8 = coeffs['RK8']
            RK9 = coeffs['RK9']
            RK10 = coeffs['RK10']
            RK11 = coeffs['RK11']
            RK12 = coeffs['RK12']
            RK13 = coeffs['RK13']
            RK14 = coeffs['RK14']
            RK15 = coeffs['RK15']
            RK16 = coeffs['RK16']
            RK17 = coeffs['RK17']
            RK18 = coeffs['RK18']
            RK19 = coeffs['RK19']
            RK20 = coeffs['RK20']

                        
            
            # 예측값 계산
            cpe19p_pred_x = (RK1 
            + RK3*(rx/1e6) + RK5*(-ry/1e6) 
            + RK7*(rx**2)/1e9 + RK9*(rx*ry)/1e9 + RK11*(ry**2)/1e9 
            + RK13*(rx**3)/1e12 + RK15*(rx**2*ry)/1e12 + RK17*(rx*ry**2)/1e12 + RK19*(ry**3)/1e12 )
            

            cpe19p_pred_y = (RK2 
            + RK4*(ry/1e6) + RK6*(rx/1e6)
            + RK8*(ry**2)/1e9 + RK10*(ry*rx)/1e9 + RK12*(rx**2)/1e9
            + RK14*(ry**3)/1e12 + RK16*(ry**2*rx)/1e12 + RK18*(ry*rx**2)/1e12 + RK20*(rx**3)/1e12 )

            


            
            # CPE 잔차값 계산
            cpe19p_resi_x = residual_x_depsm - cpe19p_pred_x 
            cpe19p_resi_y = residual_y_depsm - cpe19p_pred_y

            # 결과 저장
            predictions_list.append({
                'psm_fit_x': psm_fit_x,
                'psm_fit_y': psm_fit_y,
                'residual_x_depsm': residual_x_depsm,
                'residual_y_depsm': residual_y_depsm,
                'cpe19p_pred_x': cpe19p_pred_x,
                'cpe19p_pred_y': cpe19p_pred_y,
                'cpe19p_resi_x': cpe19p_resi_x,
                'cpe19p_resi_y': cpe19p_resi_y
            })
        else:
            # 해당하는 회귀계수가 없을 경우 NaN 처리
            predictions_list.append({
                'cpe19p_pred_x': np.nan,
                'cpe19p_pred_y': np.nan,
                'cpe19p_resi_x': np.nan,
                'cpe19p_resi_y': np.nan
            })
    
    # 결과를 DataFrame으로 변환
    df_predictions = pd.DataFrame(predictions_list)
    # 원본 데이터와 병합
    df_resi_depsm = pd.concat([df_residata.reset_index(drop=True), df_predictions], axis=1)
    return df_resi_depsm




        







################################### nau 파일 처리 및 데이터 저장 #####################################################################

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'nau파일 전처리 시작.')
process_nau_files(folder_path, columns_to_extract)
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'nau파일 전처리 완료.')



################################### MULTI LOT REGRESSION #####################################################################

# 데이터 불러오기
df_rawdata = pd.read_excel("output.xlsx", sheet_name='RawData-1')

# 회귀분석 수행
df_coeff = multi_lot_regression(df_rawdata)

# 결과를 엑셀 파일에 저장
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_coeff.to_excel(writer, sheet_name='OSR_K', index=False)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'OSR Regression 완료.')

################################### MULTI LOT FITTING, RESIDUAL #####################################################################

# 잔차 계산
df_predictions = multi_lot_fitting_residual(df_rawdata, df_coeff)

# 예측 결과를 엑셀 파일에 저장
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_predictions.to_excel(writer, sheet_name='OSR_raw_fit_resi', index=False)

    
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'OSR Fitting, Residual 완료.')


################################### PSM Input Decorrect #####################################################################


# 데이타 불러오기 
df_psm_input = pd.read_excel('output.xlsx', sheet_name='PerShotMRC')
df_residata = pd.read_excel('output.xlsx', sheet_name='OSR_raw_fit_resi')


# PSM Input(Shot별 RK값)을 fitting하고 residual_x에서 빼주기 
df_psm_de = psm_decorrect(df_residata, df_psm_input)


# mrc_de 결과를 엑셀 파일에 저장
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_psm_de.to_excel(writer, sheet_name='PSM_decorrect', index=False)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'PSM Decorrect 완료.' )



################################### Resi_depsm을 CPE 38para 모델링 #####################################################################

# 데이타 불러오기 
df_resi_depsm = pd.read_excel('output.xlsx', sheet_name='PSM_decorrect')


# CPE 38para 계산
df_cpe19p = resi_to_cpe(df_resi_depsm)


# 결과를 엑셀 파일에 저장
with pd.ExcelWriter("output.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_cpe19p.to_excel(writer, sheet_name='CPE_38p', index=False)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'),'CPE 38para Regression 완료.')



################################### CPE 38para를 Fitting #####################################################################

# multi_lot_cpe_fitting 함수 사용하여 예측값 계산
df_cpe38p_fit_res = cpe38p_fitting(df_resi_depsm, df_cpe19p)


# 결과를 엑셀 파일에 저장
with pd.ExcelWriter("output.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_cpe38p_fit_res.to_excel(writer, sheet_name='CPE38p_fit_res', index=False)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'CPE_38para Fitting, Residual 완료.')





print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '모든 작업이 완료')




B3N049.1_VH075040_VH075030_20240530171740_11.nau 파일 읽기 성공
B3N049.1_VH075040_VH075030_20240530171740_11.nau 파일에 새로운 시트 추가 완료
PDS211.1_WF075040_WF075030_20240716235504_11.nau 파일 읽기 성공
PDS211.1_WF075040_WF075030_20240716235504_11.nau 파일에 새로운 시트 추가 완료
PRQ328.1_WC046040_WC046030_20240430002132_11.nau 파일 읽기 성공
PRQ328.1_WC046040_WC046030_20240430002132_11.nau 파일에 새로운 시트 추가 완료
2024-10-04 00:44:38 nau파일 전처리 시작.
2024-10-04 00:44:44 nau파일 전처리 완료.
2024-10-04 00:44:50 OSR Regression 완료.
2024-10-04 00:44:57 OSR Fitting, Residual 완료.
2024-10-04 00:45:11 PSM Decorrect 완료.
2024-10-04 00:45:27 CPE 38para Regression 완료.
2024-10-04 00:45:46 CPE_38para Fitting, Residual 완료.
2024-10-04 00:45:46 모든 작업이 완료
